In [19]:
DATA_DIR = '../../../analysis/data/nbmodel_templates.csv'
BATCH_SIZE = 100
HIDDEN_DIM = 100
SEQ_LENGTH = 50
WEIGHTS = ''

GENERATE_LENGTH = 100
LAYER_NUM = 2

In [20]:
reports = []

with open(DATA_DIR, encoding='utf-8-sig') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            reports.append(row[3].replace('"', '').replace("[", '').replace(']', ''))
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are report, report_chunks, template_chunks, templates
Processed 1953 lines.


In [21]:
# data = ' '.join(reports).split(' ')
data = np.array(reports).flatten()
data = ' '.join(data)
len(data)

313725

In [22]:
data[:10]

'${player_n'

In [23]:
chars = list(set(data)) #set: gets unique values
VOCAB_SIZE = len(chars)

print('Unique Words:\n{}\n\nVOCAB_SIZE: {}'.format(chars, VOCAB_SIZE))

Unique Words:
['7', 'u', 'l', 'V', 'C', 'g', 'b', 'p', 'G', '3', 'e', 'i', 'T', 'D', 'R', ',', 'r', 'J', '0', '8', 'v', 'h', '1', 'y', 'X', '(', 'x', '_', 'd', '-', 'H', '5', '4', 't', 'B', 'W', 'w', 'I', 'E', '6', 'M', 'N', 'k', 'o', 'L', 'A', 'F', 's', '$', 'n', 'K', 'S', 'q', '}', '.', 'P', 'a', 'f', 'U', '{', '2', 'j', ')', 'z', 'm', '9', 'O', 'c', "'", ' ']

VOCAB_SIZE: 70


In [24]:
idx_to_char = {i: char for i, char in enumerate(chars)}
char_to_idx = {char: i for i, char in enumerate(chars)}

In [25]:
import numpy as np

SEQ_LENGTH = 60 #input sequence length
N_FEATURES = VOCAB_SIZE #one hot encoding here, that's why, but deduplicated for clarity

N_SEQ = int(np.floor((len(data) - 1) / SEQ_LENGTH))

X = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))
y = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))

for i in range(N_SEQ):
  X_sequence = data[i * SEQ_LENGTH: (i + 1) * SEQ_LENGTH]
  X_sequence_ix = [char_to_idx[c] for c in X_sequence]
  input_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
  for j in range(SEQ_LENGTH):
    input_sequence[j][X_sequence_ix[j]] = 1. #one-hot encoding of the input characters
  X[i] = input_sequence
  
  y_sequence = data[i * SEQ_LENGTH + 1: (i + 1) * SEQ_LENGTH + 1] #shifted by 1 to the right
  y_sequence_ix = [char_to_idx[c] for c in y_sequence]
  target_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
  for j in range(SEQ_LENGTH):
    target_sequence[j][y_sequence_ix[j]] = 1. #one-hot encoding of the target characters
  y[i] = target_sequence

In [26]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, TimeDistributed, Dense, Activation

# constant parameter for the model
HIDDEN_DIM = 700 #size of each hidden layer, "each layer has 700 hidden states"
LAYER_NUM = 2 #number of hidden layers, how much were used?

model = Sequential()
model.add(LSTM(HIDDEN_DIM, 
               input_shape=(None, VOCAB_SIZE), 
               return_sequences=True))
for _ in range(LAYER_NUM - 1):
  model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [33]:
def generate_text(model, length):
  ix = [np.random.randint(VOCAB_SIZE)]
  y_char = [idx_to_char[ix[-1]]]
  X = np.zeros((1, length, VOCAB_SIZE))
  for i in range(length):
    X[0, i, :][ix[-1]] = 1.
    ix = np.argmax(model.predict(X[:, :i+1,:])[0], 1)
    print(idx_to_char[ix[-1]], end="")
    y_char.append(idx_to_char[ix[-1]])
#   return ''.join(y_char)

In [28]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
# callback to save the model if better
filepath="tgt_model.hdf5"
save_model_cb = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callback to stop the training if no improvement
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=4)
# callback to generate text at epoch end
class generateText(Callback):
    def on_epoch_end(self, batch, logs={}):
        pass
#         print(generate_text(self.model, 100))
        
generate_text_cb = generateText()

callbacks_list = [save_model_cb, early_stopping_cb, generate_text_cb]

In [29]:
NB_EPOCHS = 200 #max number of epochs to train, "200 epochs"
BATCH_SIZE = 128 
VALIDATION_SPLIT = 0.1 #proportion of the batch used for validation at each epoch

model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=NB_EPOCHS, 
          callbacks=callbacks_list, validation_split=VALIDATION_SPLIT)

Train on 4705 samples, validate on 523 samples
Epoch 1/200
4705/4705 [==============================] - 27s 6ms/step - loss: 3.2795 - acc: 0.1306 - val_loss: 2.9392 - val_acc: 0.1862

Epoch 00001: val_acc improved from -inf to 0.18620, saving model to tgt_model.hdf5
Epoch 2/200
4705/4705 [==============================] - 24s 5ms/step - loss: 2.6587 - acc: 0.2361 - val_loss: 2.2135 - val_acc: 0.3458

Epoch 00002: val_acc improved from 0.18620 to 0.34579, saving model to tgt_model.hdf5
Epoch 3/200
4705/4705 [==============================] - 24s 5ms/step - loss: 1.7611 - acc: 0.4794 - val_loss: 1.2169 - val_acc: 0.6849

Epoch 00003: val_acc improved from 0.34579 to 0.68489, saving model to tgt_model.hdf5
Epoch 4/200
4705/4705 [==============================] - 24s 5ms/step - loss: 1.0912 - acc: 0.6938 - val_loss: 1.1484 - val_acc: 0.6095

Epoch 00004: val_acc did not improve from 0.68489
Epoch 5/200
4705/4705 [==============================] - 24s 5ms/step - loss: 0.7045 - acc: 0.8089 -

In [38]:
generate_text(model, 200)

er the ${opp}. ${player_name} caught ${receptions} of ${rec_targets} targets for ${rec_yards} yards in the ${team}' week ${week} win over the ${opp}. ${player_name} caught ${receptions} of ${rec_targe